# Hydro estimator into in NetCDF 

This notebook is to:
* convert hydro estimator data (1-hr accumulated) in rainfall rate (mm/s) every 3-hr
* convert ascii to netcdf

In [2]:
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import itertools
import sys

In [3]:
# !gunzip ../example_data/Hydro_Estimator/*.gz

### Extract lat, lon data for ASCII

In [4]:
%%time
latlon = np.loadtxt('../example_data/Hydro_Estimator/zworld.latlon2', delimiter="\t", dtype=str)

CPU times: user 2min 54s, sys: 7.46 s, total: 3min 2s
Wall time: 3min


In [5]:
%%time
latlon_values = np.array(' '.join(latlon).split(' '))
latlon_values = list(filter(None, latlon_values))

CPU times: user 1min 9s, sys: 17.1 s, total: 1min 26s
Wall time: 1min 25s


In [6]:
lat_all = latlon_values[0:-1:2]
lon_all = latlon_values[1::2]

In [7]:
lat_hydro = np.array(lat_all, dtype=np.float64)
lon_hydro = np.array(lon_all, dtype=np.float64)

### Extract values from Hydro Estimator ASCII and convert to Rainfall rate (mm/s)

In [8]:
hour = 3  
datetime = 201908312
datetime_comb = []
filenames = []

In [9]:
if datetime%100 != 0:
    datetime_comb[0:hour] = list(range(datetime - 1*(hour-1), datetime + 1, 1))
else:
    datetime_24 = datetime - 76
    datetime_comb[0:(hour-1)] = list(range(datetime_24 - 1*(hour-1), datetime_24, 1))
    datetime_comb.append(datetime)

In [10]:
# Check the datetime 
print('Now you are calculating RAINRATE (mm/s) on: ' + str(datetime_comb))

Now you are calculating RAINRATE (mm/s) on: [201908310, 201908311, 201908312]


In [11]:
for i in range(0,hour):
    filenames.append('../example_data/Hydro_Estimator/world1hr.' + str(datetime_comb[i]))

In [12]:
input_file = []

In [13]:
%%time
for i in range(0,hour):
    input_file.append([np.loadtxt(filenames[i], delimiter='\t', skiprows=1)])
    
# input_file_3 = np.loadtxt(filename_3, delimiter='\t', skiprows=1)
# input_file_2 = np.loadtxt(filename_2, delimiter='\t', skiprows=1)
# input_file_1 = np.loadtxt(filename_1, delimiter='\t', skiprows=1)

CPU times: user 9min 58s, sys: 4.08 s, total: 10min 2s
Wall time: 10min 2s


In [14]:
for i in range(0,hour):
    input_file[i] = [y for x in input_file[i] for y in x]

Calculating the Rainfall with equation, R = (value-2) * 0.3048. ref: https://www.star.nesdis.noaa.gov/smcd/emb/ff/digGlobalData.php  
Since GLDAS is every 3-hour, we want the rainfall rate at the 00, 03, 06, etc. that is counting the 

In [15]:
rain = []
for i in range(0,hour):
    rain.append([(np.asanyarray(input_file[i])-2)*0.3048])

RAINRATE_hydro = [sum(x)/10800 for x in zip(*rain)]   #Conver into mm/s)

### Create NetCDF file

In [16]:
# NC file setup
Hydro_estimator = Dataset('../example_data/Hydro_Estimator_netcdf/Hydro_Estimator' + str(datetime) + '.nc','w', format='NETCDF4') 

In [17]:
# Set dimention and lat lon
Hydro_estimator.description = 'Hydro Estimator RAINRATE'
## create the lat and lon dimensions.
Hydro_estimator.createDimension('Time')
Hydro_estimator.createDimension('south_north',3111)
Hydro_estimator.createDimension('west_east',8001)

<class 'netCDF4._netCDF4.Dimension'>: name = 'west_east', size = 8001

In [18]:
## create the RAINRATE variable
lat = Hydro_estimator.createVariable('lat','float64',('Time','south_north','west_east'))
lon = Hydro_estimator.createVariable('lon','float64',('Time','south_north','west_east'))
RAINRATE = Hydro_estimator.createVariable('RAINRATE','float64',('Time','south_north','west_east'))
## set the units attribute.
RAINRATE.units =  'mm/s'
## write data to variables.
lat[1,:len(lat_hydro),:len(lon_hydro)] = lat_hydro.reshape(3111,8001)
lon[1,:len(lat_hydro),:len(lon_hydro)] = lon_hydro.reshape(3111,8001)
RAINRATE_hydro = np.asarray(RAINRATE_hydro)
RAINRATE[1,:len(lat_hydro),:len(lon_hydro)] = RAINRATE_hydro.reshape(3111,8001)

In [19]:
Hydro_estimator.close()